# Librerías

In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
import joblib

import a_funciones as fn

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

#Para redes
import tensorflow as tf
from keras_tuner.tuners import RandomSearch

In [ ]:
# Cargar los datos escalados, la variable de respuesta y el scaler
df_historicos_scaled = joblib.load('salidas/df_historicos_transf.pkl')
y = joblib.load('salidas/v.respuesta.pkl')
sc = joblib.load('salidas/scaler.pkl')

In [ ]:
df_historicos_scaled.info()

In [ ]:
df_new_credit = pd.read_csv('data/datos_nuevos_creditos.csv')
df_new_credit.head()

In [ ]:
df_new_credit.info()

In [ ]:
monto_credito = df_new_credit['NewLoanApplication']

In [ ]:
del df_new_credit['NewLoanApplication']
del df_new_credit['ID']

In [ ]:
df_new_credit.head()

In [ ]:
df_newcredit_sc, _ = fn.procesar_datos(df_new_credit, scaler=sc)
df_newcredit_sc.info()

# Segmentación

Realizar selección de variables con PCA

In [ ]:
#variables con respecto a las variables y componentes

kmeans = KMeans(n_clusters=3, random_state=42)
df_historicos_scaled['Segmento'] = kmeans.fit_predict(df_historicos_scaled)

In [ ]:
# Aplicar PCA para reducir las dimensiones a 2
pca = PCA(n_components=2)
df_historicos_scaled_pca = pca.fit_transform(df_historicos_scaled.drop(columns=['Segmento']))

plt.figure(figsize=(8, 6))
plt.scatter(df_historicos_scaled_pca[:, 0], df_historicos_scaled_pca[:, 1], c=df_historicos_scaled['Segmento'], cmap='viridis', s=50, alpha=0.6)

plt.title('Segmentos de Clientes usando KMeans')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')

plt.colorbar(label='Segmento')
plt.show()

# Train / Test

In [ ]:
from sklearn.model_selection import train_test_split
x = df_historicos_scaled 

In [ ]:
# Separar datos
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

#Imprimir Tamaño de dataset, corresponden al 80,20
print("Tamaño del conjunto de entrenamiento. X: ", X_train.shape," Y: ", y_train.shape)
print("Tamaño del conjunto de validación. X: ", X_test.shape," Y: ", y_test.shape )


# Modelos

## Redes neuronales

In [ ]:
regression_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [ ]:
# Configurar el optimizador y la función de pérdida para un problema de regresión
regression_model.compile(optimizer='adam', loss='mse', metrics=[MeanSquaredError(), RootMeanSquaredError()])

# Entrenar el modelo usando el optimizador y la arquitectura definidas
regression_model.fit(X_train, y_train, batch_size=100, epochs=10, validation_data=(X_test, y_test))

# Evaluar el modelo
test_loss, test_mse, test_rmse = regression_model.evaluate(X_test, y_test, verbose=2)

print("Test Mean Squared Error:", test_mse)
print("Test Root Mean Squared Error:", test_rmse)

In [ ]:
# Definir la función de creación del modelo
def build_model(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=X_train.shape[1:]))

    # Añadir capas ocultas con hiperparámetros para el número de unidades y la activación
    for i in range(hp.Int('num_layers', 1, 6)):
        model.add(tf.keras.layers.Dense(
            units=64,  # Número fijo de unidades en cada capa
            activation=hp.Choice('activation', ['relu', 'tanh']),
            kernel_initializer=hp.Choice('init_mode', ['uniform', 'he_normal'])
        ))

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compilar el modelo con hiperparámetros para el optimizador
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=[tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.RootMeanSquaredError()]
    )

    return model

In [ ]:
# Crear el tuner
tuner = RandomSearch(
    build_model,
    objective='val_mean_squared_error',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='helloworld'
)

# Realizar la búsqueda de hiperparámetros
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=100)

# Obtener los mejores hiperparámetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


# Mostrar los mejores hiperparámetros
print("Best Hyperparameters:")
print(f"- Number of layers: {best_hps.get('num_layers')}")


In [ ]:
# Hacer predicciones con el modelo entrenado
y_pred_nuevos = regression_model.predict(df_newcredit_sc)

In [ ]:
## Residuales

# Obtener las predicciones del modelo en los datos de prueba
y_pred = regression_model.predict(X_test).flatten()

# Calcular los residuales
residuales = y_test - y_pred

# Histograma de los residuales
plt.figure(figsize=(8, 6))
plt.hist(residuales, bins=20, edgecolor='k')
plt.title('Histograma de Residuales')
plt.xlabel('Residual')
plt.ylabel('Frecuencia')
plt.show()

# Gráfico de dispersión de los residuales vs. predicciones
plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuales, alpha=0.5)
plt.title('Residuales vs. Predicciones')
plt.xlabel('Predicciones')
plt.ylabel('Residuales')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

# Gráfico de dispersión de los residuales vs. el valor verdadero
plt.figure(figsize=(8, 6))
plt.scatter(y_test, residuales, alpha=0.5)
plt.title('Residuales vs. Valor Verdadero')
plt.xlabel('Valor Verdadero')
plt.ylabel('Residuales')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Crear el histograma de las predicciones
plt.figure(figsize=(8, 4))
plt.hist(y_pred_nuevos, bins=20, color='skyblue', edgecolor='black')
plt.title('Histograma de Predicciones')
plt.xlabel('Valor Predicho')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()

In [ ]:
df_predic = df_new_credit.copy()
df_predic['NoPaidPer'] = y_pred_nuevos
df_predic

In [ ]:
# Filtrar las predicciones que están entre 0 y 0.1
tasa1 = df_predic.loc[(df4['predicciones'] >= 0) & (df_predic['predicciones'] < 0.15)]
# Filtrar las predicciones que están entre 0 y 0.1
tasa2 = df_predic.loc[(df_predic['predicciones'] >= 0.15)]

print(tasa1.shape)
print(tasa2.shape)